In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import rasterio as rio
import datetime
import os
import numpy as np

In [174]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        
        self.conv1_1 = nn.Conv2d(6, 64, 3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, 1, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, 1, padding=0)
        
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, 1, padding=0)
        
        self.up_5 = nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1)
        self.up_4 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
        self.up_3 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.up_2 = nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1)
        self.up_1 = nn.ConvTranspose2d(32, 1, 4, stride=2, padding=1)
        
    def forward(self, x):   
        # 224x224
        x = x.cuda()
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.max_pool2d(x, 2, 2)
        # 112x112
        
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = F.max_pool2d(x, 2, 2)
        # 56x56
        
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = F.max_pool2d(x, 2, 2)
        # 28x28
        
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = F.max_pool2d(x, 2, 2)
        # 14x14
        
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = F.max_pool2d(x, 2, 2)
        # 7x7
        
        x = F.relu(self.up_5(x))
        x = F.relu(self.up_4(x))
        x = F.relu(self.up_3(x))
        x = F.relu(self.up_2(x))
        x = F.relu(self.up_1(x))
        return x

In [219]:
n_epochs = 10
learning_rate = 0.00001
momentum = 0.5
print_interval = 1000

random_seed = 1
torch.manual_seed(random_seed)

In [220]:
net = VGG16().cuda()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [177]:
def test():
    net.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(f'Test set: Avg. loss: {test_loss}')

In [178]:
#test()
#for epoch in (0, n_epochs):
#    train(epoch)
#    test()

In [179]:
file = rio.open('./full_band.tif')
data = file.read(); print(data.shape)
X = torch.tensor(data[:, 0:224, 0:224], dtype=torch.float); print(X.shape)

(6, 1986, 1551)
torch.Size([6, 224, 224])


In [180]:
X = X.view(1, 6, 224, 224).cuda()

In [181]:
net.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    output = net(X)

In [182]:
output.shape

torch.Size([1, 1, 224, 224])

In [183]:
net.train()

VGG16(
  (conv1_1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d(512, 512, kernel_size=(

In [184]:
from torch.utils.data import Dataset, DataLoader
import rasterio as rio
from rasterio.plot import reshape_as_image

class ChampaignRaster(Dataset):
    def __init__(self, split='train'):
        super().__init__()
        
        
        self.X = os.listdir('./data/vgg16/X')
        self.y = os.listdir('./data/vgg16/y')
        size = int(len(self.X) * 0.7)
        
        if split == 'train':
            self.X = self.X[0:size]
            self.y = self.y[0:size]
        else:
            self.X = self.X[size:len(self.X)]
            self.y = self.y[size:len(self.y)]
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        x = self.X[i]
        rst_x = rio.open('./data/vgg16/X/' + x)
        band_x = rst_x.read()
        tensor_x = torch.from_numpy(band_x)
        tensor_x = tensor_x.type(torch.cuda.FloatTensor)
        #tensor_x = Variable(tensor_x, requires_grad=True).cuda()
        
        y = self.y[i]
        rst_y = rio.open('./data/vgg16/y/' + y)
        band_y = rst_y.read()
        tensor_y = torch.from_numpy(band_y)
        tensor_y = tensor_y.type(torch.cuda.FloatTensor)
        #tensor_y = Variable(tensor_y, requires_grad=True).cuda()
        return tensor_x, tensor_y
        

In [227]:
s = ChampaignRaster()

train_dataset = ChampaignRaster(split='train')
val_dataset = ChampaignRaster(split='val')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=10)
val_loader = DataLoader(val_dataset, batch_size=10)
#train_dataset.__getitem__(0)

In [228]:
train_dataset[1][0].shape, train_dataset[1][1].shape

(torch.Size([6, 224, 224]), torch.Size([1, 224, 224]))

In [238]:
objective = nn.BCELoss().cuda()
def train():
    net.train()
    for idx, (data, target) in enumerate(train_loader):
        
#         print(data[0, 0, 1:3, 1:3])
        #print(data.shape)
        optimizer.zero_grad()
        output = torch.sigmoid(net(data))
#         print(output)
        loss = objective(output.reshape(10, -1), target.reshape(10, -1))
#         print(loss.item())
        loss.backward()
#         for param in net.parameters():
#             print(param.grad)
        optimizer.step()
        if idx % 100 == 0:
            print(f'Loss: {loss.item()}')
        break

In [239]:
train()

tensor([[[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          ...,
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]]],


        [[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          ...,
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]]]],
       device='cuda:0', grad_fn=<SliceBackward>) tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0